In [25]:
import tensorflow as tf
import pickle
import os
import shutil
import tempfile
import datetime
from glob import iglob
import time
# import attention
from collections import deque
import pickle
import pyreader
import numpy as np
import csv
from batchmake import Batcher
import pprint
tf.set_random_seed(777)
from tensorflow.python.layers import core

In [26]:
datas= 'data_samples/'
vocabulary = 'vocab.csv'
vocabs = []
with open(vocabulary, 'r', newline='', encoding='utf-8') as vocab:
    words = csv.reader(vocab)
    for i, word in enumerate(words):
        vocabs.append(word[0])
#         vocabs[word[0]]=i
#         vocabs[word] = i
print(len(vocabs))
word_to_id = {word: i for i, word in enumerate(vocabs)}
id_to_word = {i:word for i, word in enumerate(vocabs)}

22673


In [27]:
id_to_word[0], word_to_id['<PAD>']

('<PAD>', 0)

In [28]:
hidden_size=32
sequence=5
embedding_dim=50
attention_size = 50
batch_size=16
vocab_size=len(vocabs)
checkpoint = './seq2seq_test.ckpt'

In [29]:
def source_to_seq(text):
    '''Prepare the text for the model'''
    sequence_length = 5
    return [word_to_id.get(word, word_to_id['<UNK>']) for word in text]+ [word_to_id['<PAD>']]*(sequence_length-len(text))

In [35]:
input_sentence = 'session'
text = source_to_seq([input_sentence])

In [39]:
text = [1, 0, 0, 0, 2]

In [32]:
lst = [3,4,2]* 3
lst

[3, 4, 2, 3, 4, 2, 3, 4, 2]

In [42]:
checkpoint = './seq2seq_test/seq2seq_test.ckpt'
loaded_graph = tf.Graph()

with tf.Session(graph=loaded_graph) as Ses:
    loader = tf.train.import_meta_graph(checkpoint +'.meta')
    loader.restore(Ses, checkpoint)
    
    print('loaded_graph.get_all_collection_keys()',loaded_graph.get_all_collection_keys())
    input_data = loaded_graph.get_tensor_by_name('inputs_xdata:0')
    logits = loaded_graph.get_tensor_by_name('predictions:0')
    source_sequence_length = loaded_graph.get_tensor_by_name('source_sequence_length:0')
    target_sequence_length = loaded_graph.get_tensor_by_name('target_sequence_length:0')

    answer_logits = Ses.run(logits, {input_data: [text]*batch_size, 
                                      target_sequence_length: [len(text)]*batch_size, 
                                      source_sequence_length: [len(text)]*batch_size})
    
pad = word_to_id['<PAD>']

print('Original Text:', input_sentence)

print('\nSource')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format(" ".join([id_to_word[i] for i in text])))


INFO:tensorflow:Restoring parameters from ./seq2seq_test/seq2seq_test.ckpt
loaded_graph.get_all_collection_keys() ['trainable_variables', 'variables', 'model_variables', 'while_context', 'cond_context', 'train_op', 'queue_runners', 'summaries']
Original Text: session

Source
  Word Ids:    [1, 0, 0, 0, 2]
  Input Words: <UNK> <PAD> <PAD> <PAD> <GO>


In [41]:
answer_logits
# print('\nTarget')
# print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
# print('  Response Words: {}'.format(" ".join([id_to_word[i] for i in answer_logits if i != pad])))

array([[5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5],
       [5, 5, 5, 5, 5]])